In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
from pathlib import Path

2023-10-19 17:19:23.398787: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
DATA_PATH = Path("/scratch/ajb5d/ecg/tfrecords/")
TRAIN_RECS = list(DATA_PATH.glob("train*.tfrecords"))
VAL_RECS = list(DATA_PATH.glob("train*.tfrecords"))

In [3]:
BATCH_SIZE = 64

record_format = {
    'ecg/data': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'age': tf.io.FixedLenFeature([], tf.float32),
    'gender': tf.io.FixedLenFeature([], tf.int64),
}

def _parse_record(record):
    example = tf.io.parse_single_example(record, record_format)
    ecg_data = tf.reshape(example['ecg/data'], [5000,12])
    label = example['gender']
    return ecg_data, label

def load_dataset(filenames):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(_parse_record, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset

def get_dataset(filenames, labeled=True):
    dataset = load_dataset(filenames)
    dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

In [4]:
from datetime import datetime
import os

def make_checkpoint_dir(data_path, label):
    current_datetime = datetime.now()
    formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")
    output_dir = f"{label}-{formatted_datetime}"
    output_path = f"{data_path}/{output_dir}"
    
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    return output_path

In [5]:
train_dataset = get_dataset(TRAIN_RECS)
val_dataset = get_dataset(VAL_RECS)

2023-10-19 17:19:28.704532: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1636] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46594 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:41:00.0, compute capability: 8.6


In [6]:
## Resnet from https://github.com/antonior92/automatic-ecg-diagnosis/tree/master
## https://www.nature.com/articles/s41467-020-15432-4

def residual_unit(x, y, n_samples_out, n_filters_out, prefix, kernel_size = 16):
    n_samples_in = y.shape[1]
    downsample = n_samples_in // n_samples_out
    n_filters_in = y.shape[2]
    
    if downsample == 1:
        y = y
    else:
        y = layers.MaxPooling1D(downsample, strides=downsample, padding='same', name = f"{prefix}_mp_opt")(y)
        
    if n_filters_in != n_filters_out:
        y = layers.Conv1D(n_filters_out, 1, padding='same', use_bias=False, name = f"{prefix}_conv_opt")(y)
        
    x = layers.Conv1D(n_filters_out, kernel_size, padding='same', use_bias=False, name = f"{prefix}_conv1")(x)
    x = layers.BatchNormalization(name = f"{prefix}_bn1")(x)
    x = layers.Activation("relu", name = f"{prefix}_act1")(x)
    x = layers.Dropout(0.2, name = f"{prefix}_dropout1")(x)
    x = layers.Conv1D(n_filters_out, kernel_size, strides=downsample, padding='same', use_bias=False, name = f"{prefix}_conv2")(x)

    x = layers.Add(name = f"{prefix}_add")([x,y])
    y = x
    x = layers.BatchNormalization(name = f"{prefix}_bn2")(x)
    x = layers.Activation("relu", name = f"{prefix}_act2")(x)
    x = layers.Dropout(0.2, name = f"{prefix}_dropout2")(x)
    return (x,y)
    
    
    

input_layer = tf.keras.layers.Input(shape=(5000,12), name="input")
x = input_layer
x = layers.Conv1D(64, 16, padding='same', use_bias=False, name = "conv_1")(x)
x = layers.BatchNormalization(name="bn")(x)
x = layers.Activation("relu", name="relu")(x)

x, y = residual_unit(x,x,1024,128, "res1")
x, y = residual_unit(x,y,256,196, "res2")
x, y = residual_unit(x,y,64,256, "res3")
x, _ = residual_unit(x,y,16,320, "res4")

x = tf.keras.layers.Flatten(name="flatten")(x)
x = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(x)

model = tf.keras.models.Model(input_layer, x)

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy', tf.keras.metrics.AUC()]
)

In [7]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 5000, 12)]           0         []                            
                                                                                                  
 conv_1 (Conv1D)             (None, 5000, 64)             12288     ['input[0][0]']               
                                                                                                  
 bn (BatchNormalization)     (None, 5000, 64)             256       ['conv_1[0][0]']              
                                                                                                  
 relu (Activation)           (None, 5000, 64)             0         ['bn[0][0]']                  
                                                                                              

In [8]:
callbacks = [
    tf.keras.callbacks.TerminateOnNaN(),
    tf.keras.callbacks.ReduceLROnPlateau(),
    tf.keras.callbacks.ModelCheckpoint(make_checkpoint_dir("data/models", "resnet-gender"))
]

model.fit(train_dataset, epochs=10, validation_data=val_dataset, callbacks=callbacks)

Epoch 1/10


2023-10-19 17:19:33.982104: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8904
2023-10-19 17:19:35.707869: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:625] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-10-19 17:19:35.735482: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f2628ce0780 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-19 17:19:35.735527: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2023-10-19 17:19:35.817991: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-19 17:19:36.371357: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.

   7851/Unknown - 554s 69ms/step - loss: 0.5016 - accuracy: 0.7579 - auc: 0.8375INFO:tensorflow:Assets written to: data/models/resnet-gender-2023-10-19_17-19-30/assets


INFO:tensorflow:Assets written to: data/models/resnet-gender-2023-10-19_17-19-30/assets


7851/7851 [==============================] - 687s 86ms/step - loss: 0.5016 - accuracy: 0.7579 - auc: 0.8375 - val_loss: 0.5552 - val_accuracy: 0.7491 - val_auc: 0.8395 - lr: 0.0010
Epoch 2/10
7851/7851 [==============================] - ETA: 0s - loss: 0.4371 - accuracy: 0.7963 - auc: 0.8787INFO:tensorflow:Assets written to: data/models/resnet-gender-2023-10-19_17-19-30/assets


...

7851/7851 [==============================] - 670s 85ms/step - loss: 0.3372 - accuracy: 0.8530 - auc: 0.9302 - val_loss: 0.4747 - val_accuracy: 0.7925 - val_auc: 0.9021 - lr: 0.0010
Epoch 9/10
7850/7851 [============================>.] - ETA: 0s - loss: 0.3230 - accuracy: 0.8599 - auc: 0.9360INFO:tensorflow:Assets written to: data/models/resnet-gender-2023-10-19_17-19-30/assets


INFO:tensorflow:Assets written to: data/models/resnet-gender-2023-10-19_17-19-30/assets


7851/7851 [==============================] - 669s 85ms/step - loss: 0.3230 - accuracy: 0.8599 - auc: 0.9360 - val_loss: 0.4409 - val_accuracy: 0.8120 - val_auc: 0.9067 - lr: 0.0010
Epoch 10/10
7850/7851 [============================>.] - ETA: 0s - loss: 0.3091 - accuracy: 0.8671 - auc: 0.9415INFO:tensorflow:Assets written to: data/models/resnet-gender-2023-10-19_17-19-30/assets


INFO:tensorflow:Assets written to: data/models/resnet-gender-2023-10-19_17-19-30/assets


7851/7851 [==============================] - 671s 85ms/step - loss: 0.3091 - accuracy: 0.8671 - auc: 0.9415 - val_loss: 0.4095 - val_accuracy: 0.8267 - val_auc: 0.9128 - lr: 0.0010


In [9]:
model.save("data/models/resnet-gender.keras")